In [1]:
import re,os
def convertToRegexIncludes(path,prePathRegex):
    (folder,fileName) = os.path.split(path);
    mo = re.match(prePathRegex, path);
    if(mo):
        s = mo.groups(1)[0];
        fileName2 = fileName.replace(".",r"\."); # replace dots
        replaceString = [
            "s@%s@%s@g" % (r'#include\s*\".*?/'+fileName2+r'\"', r'#include \"' + s + r'\"')
            ]
        #print("Change :", fileName,"-->", s)
        return fileName,replaceString;
        
    else:
        raise NameError("No Match: ", mo)
        
def convertToRegexCMAKE(path,prePathRegex):
    (folder,fileName) = os.path.split(path);
    mo = re.match(prePathRegex, path);
    if(mo):
        s = mo.groups(1)[0];
        fileName2 = fileName.replace(".",r"\."); # replace dots
        replaceString = [
            r"s@%s@%s@g" % (r'(src|include).*?/'+fileName2 , r'\1/' + s ),
            ]
        #print("Change :", fileName,"-->", s)
        return fileName,replaceString;
        
    else:
        raise NameError("No Match: ", mo)
        
def makeFoldersSubCaps(globbingExp,exceptFolder,exceptRename,relTo):
    renames=[]
    folders = []
    for glob in globbingExp:
        folders += glob2.glob(glob)
        
    # filter with regex
    for reg in exceptFolder:
        l = lambda f : not re.match(reg,os.path.realpath(f))
        folders= list(filter( l , folders))
    
    folders = [ os.path.relpath(f, relTo).split("/") for f in folders ]
    # sort longest path first because we mv folders
    folders.sort(key= lambda x: len(x))
    folders.reverse()
    
    
    for dirs in folders:
            
        # loop over all dirs
        origPath= os.path.join(*dirs)
        # change last dir
        change=True
        for reg in exceptRename:
            m = re.match(reg,dirs[-1])
            if m:
                change = False
                break;
        if change:
            dirs[-1] = dirs[-1][0].lower() + dirs[-1][1:]

        newPath = os.path.join(*dirs)
            
        if  origPath != newPath :
            renames.append("git mv %s %s" % (origPath, newPath))
            
    return renames

In [79]:
convertToRegex("States/StateRecorder.hpp","(.*)")

('StateRecorder.hpp',
 ['s@#include\\s*\\".*?/StateRecorder\\.hpp\\"@#include \\"States/StateRecorder.hpp\\"@g',
  's@(include|src).*?/StateRecorder\\.hpp@\\1/States/StateRecorder.hpp@g'])

In [2]:
def makeReplacements(rootDirs, func=convertToRegexIncludes):
    replacements =[];
    fileNames = set();
    for rootDir in rootDirs:
        for dirName, subdirList, fileList in os.walk(rootDir):
            relDir = os.path.relpath(dirName, rootDir)
            for f in fileList:
                filepath = os.path.join(relDir, f)
                fileName, RegexS = func(filepath,"(.*)");
                if( fileName not in fileNames):
                    fileNames.add(fileName)
                    replacements += RegexS;
    return replacements


def makeReplacementsIncludeGuard(rootDirs):
    commands =[];
    for rootDir in rootDirs:
        for dirName, subdirList, fileList in os.walk(rootDir):
            relDir = os.path.relpath(dirName, rootDir)
            for f in fileList:
                absfilepath = os.path.join(dirName, f)
                relfilepath = os.path.join(relDir, f)
                
                newInclude= relfilepath.replace(".","_")
                newInclude= newInclude.replace("/","_")
                
                regex = r'"s@#\s*ifndef\s*(\w*)\s*\n\s*#\s*define\s*(\w*)@#ifndef %s\n#define %s@s"' % (newInclude, newInclude)
                command = r'fileRegexReplace -f -r %s %s' % (regex, os.path.abspath(absfilepath))
                commands.append(command)
    return commands

Granular RigidBody Simulation Framework
===

In [159]:

# rename all folders to smallcaps
globs = ["../../**/"]


commands = makeFoldersSubCaps(globs,
                              exceptFolder=[".*?media",".*?external",".*?Doc",".*?Simulations"], 
                              exceptRename=["external", "GRSF", "LMatrixGeneration"], 
                              relTo="../")

f = open("../MoveFolders.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
for c in commands:
    f.write(c + "\n");
f.close();

In [133]:
dirs = ["../../common/**/*", 
        "../../projects/GeneralRigidBodySimulation/include/",
        "../../projects/GeneralRigidBodySimulationMPI/include/",
        "../../projects/GeneralRigidBodySimulationNoGUI/include/",
        "../../projects/Converter/include/"]

# #include replacements
replacements = makeReplacements(dirs)
#print(replacements)
s = [r'-r "%s" ' %s + "  \\\n" for s in replacements]
regex = " ".join(s);
command = r'fileRegexReplace -f %s -R ".*\.(hpp|cpp|icc)" -e "*/external/*" /home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework/'  % regex

f = open("../ReplaceIncludes.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(command);
f.close();

# cmake replacements (src and includes)
dirs = ["../../CommonSource/include", 
        "../../Projects/GeneralRigidBodySimulation/include/",
        "../../Projects/GeneralRigidBodySimulationMPI/include/",
        "../../Projects/GeneralRigidBodySimulationNoGUI/include/",
        "../../Projects/Converter/include/",
        "../../CommonSource/src", 
        "../../Projects/GeneralRigidBodySimulation/src/",
        "../../Projects/GeneralRigidBodySimulationMPI/src/",
        "../../Projects/GeneralRigidBodySimulationNoGUI/src/",
        "../../Projects/Converter/src/"]

replacements = makeReplacements(dirs,convertToRegexCMAKE)
#print(replacements)
s = [r'-r "%s" ' %s + "  \\\n" for s in replacements]
regex = " ".join(s);
command = r'fileRegexReplace -f %s -R ".*CMakeLists.txt" -e "*/external/*" /home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework/'  % regex

f = open("../ReplaceIncludesCmake.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(command);
f.close();



In [72]:
glob2.glob("../../Projects/**/")

['../../Projects/',
 '../../Projects/GeneralRigidBodySimulationMPI/',
 '../../Projects/GeneralRigidBodySimulation/',
 '../../Projects/Converter/',
 '../../Projects/media/',
 '../../Projects/GeneralRigidBodySimulationNoGUI/',
 '../../Projects/GeneralRigidBodySimulationMPI/include/',
 '../../Projects/GeneralRigidBodySimulationMPI/src/',
 '../../Projects/GeneralRigidBodySimulationMPI/resources/',
 '../../Projects/GeneralRigidBodySimulationMPI/include/GRSF/',
 '../../Projects/GeneralRigidBodySimulationMPI/include/GRSF/Common/',
 '../../Projects/GeneralRigidBodySimulationMPI/include/GRSF/ConfigFiles/',
 '../../Projects/GeneralRigidBodySimulationMPI/include/GRSF/Singeltons/',
 '../../Projects/GeneralRigidBodySimulationMPI/include/GRSF/States/',
 '../../Projects/GeneralRigidBodySimulationMPI/include/GRSF/Dynamics/',
 '../../Projects/GeneralRigidBodySimulationMPI/include/GRSF/Systems/',
 '../../Projects/GeneralRigidBodySimulationMPI/include/GRSF/States/SimulationManager/',
 '../../Projects/Gen

In [9]:
dirs = ["../../CommonSource/include", 
        "../../Projects/GeneralRigidBodySimulation/include/",
        "../../Projects/GeneralRigidBodySimulationMPI/include/",
        "../../Projects/GeneralRigidBodySimulationNoGUI/include/",
        "../../Projects/Converter/include/"]
commands = makeReplacementsIncludeGuard(dirs)
print(commands)
s = "\n".join(commands);
f = open("../ReplaceIncludeGuards.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(s);
f.close();

['fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef GRSF_Common_DemangleTypes_hpp\\n#define GRSF_Common_DemangleTypes_hpp@s" /home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework/CommonSource/include/GRSF/Common/DemangleTypes.hpp', 'fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef GRSF_Common_CPUTimer_hpp\\n#define GRSF_Common_CPUTimer_hpp@s" /home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework/CommonSource/include/GRSF/Common/CPUTimer.hpp', 'fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef GRSF_Common_SerializationHelpersTuple_hpp\\n#define GRSF_Common_SerializationHelpersTuple_hpp@s" /home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework/CommonSource/include/GRSF/Common/SerializationHelpersTuple.hpp', 'fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\

In [ ]:
## replace CmakeList.files
fileRegexReplace -r 's@\$\{(\w*)\}/(include|src)/@\${\1}/\2/GRSF/@g' -R ".*CMakeLists.txt" -e ".*external.*" \
/home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework


fileRegexReplace -r 's@\$\{(\w*)\}/(include|src)/@\${\1}/\2/GRSF/@g' -R ".*IncludeModulesSimulationFramework.cmake" -e ".*external.*" \
/home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/

ApproxMVBB
====

In [16]:
dirs = ["/home/zfmgpu/Desktop/Repository/ApproxMVBB/include",
       "/home/zfmgpu/Desktop/Repository/ApproxMVBB/src",
       "/home/zfmgpu/Desktop/Repository/ApproxMVBB/external/Diameter/include",
        "/home/zfmgpu/Desktop/Repository/ApproxMVBB/external/GeometryPredicates/include",
       "/home/zfmgpu/Desktop/Repository/ApproxMVBB/external/pugixml/include",
       "/home/zfmgpu/Desktop/Repository/ApproxMVBB/external/meta/include"]
commands = makeReplacementsIncludeGuard(dirs)
print(commands)
s = "\n".join(commands);
f = open("../ReplaceIncludeGuards.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(s);
f.close();

['fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef ApproxMVBB_KdTree_hpp\\n#define ApproxMVBB_KdTree_hpp@s" /home/zfmgpu/Desktop/Repository/ApproxMVBB/include/ApproxMVBB/KdTree.hpp', 'fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef ApproxMVBB_ContainerFunctions_hpp\\n#define ApproxMVBB_ContainerFunctions_hpp@s" /home/zfmgpu/Desktop/Repository/ApproxMVBB/include/ApproxMVBB/ContainerFunctions.hpp', 'fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef ApproxMVBB_ProjectedPointSet_hpp\\n#define ApproxMVBB_ProjectedPointSet_hpp@s" /home/zfmgpu/Desktop/Repository/ApproxMVBB/include/ApproxMVBB/ProjectedPointSet.hpp', 'fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef ApproxMVBB_ConvexHull2D_hpp\\n#define ApproxMVBB_ConvexHull2D_hpp@s" /home/zfmgpu/Desktop/Repository/ApproxMVBB/include/ApproxMVBB/ConvexHull2D.hpp', 'fileRegexReplace -f -r "s@#

In [136]:
dirs = ["../../../MinimumVolumeBox/external/Diameter/include"]
        
replacements = makeReplacements(dirs)
print(replacements)
s = [r'-r "%s"' %s for s in replacements]
regex = " ".join(s);

command = r'fileRegexReplace -f %s -R ".*\.(hpp|cpp|icc)" -e "*/external/*" /home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/MinimumVolumeBox/'  % regex
f = open("../ReplaceIncludes.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(command);
f.close();


['s@EstimateDiameter\\.hpp@Diameter/EstimateDiameter.hpp@g', 's@rand\\.h@Diameter/Utils/rand.h@g', 's@util\\.h@Diameter/Utils/util.h@g', 's@alloc\\.h@Diameter/Utils/alloc.h@g']


In [ ]:
fileRegexReplace -r 's@\$\{(\w*)\}/(include|src)/@\${\1}/\2/GMSF/@g' -R ".*CMakeLists.txt" -e ".*external.*" \
/home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/MinimumVolumeBox/


CudaFramework
===

In [6]:
dirs = ["/home/zfmgpu/Desktop/Repository/CudaFramework/include/"]
        
replacements = makeReplacements(dirs)
print(replacements)
s = [r'-r "%s"' %s for s in replacements]
regex = " \\\n".join(s);

f = open("../ReplaceIncludes.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")

command = r'fileRegexReplace -f %s -R ".*\.(hpp|cpp|icc|cu|cuh)" -e "*/external/*" /home/zfmgpu/Desktop/Repository/CudaFramework/src'  % regex
f.write(command);
f.write('\n')
command = r'fileRegexReplace -f %s -R ".*\.(hpp|cpp|icc|cu|cuh)" -e "*/external/*" /home/zfmgpu/Desktop/Repository/CudaFramework/include'  % regex
f.write(command);

command = r'fileRegexReplace -f %s -R ".*\.(hpp|cpp|icc|cu|cuh)" -e "*/external/*" /home/zfmgpu/Desktop/Repository/CudaFramework/projects'  % regex
f.write(command);

f.close();

['s@#include\\s*\\"CudaMatrix\\.hpp@#include \\"CudaFramework/CudaModern/CudaMatrix.hpp@g', 's@#include\\s*\\"CudaAlloc\\.hpp@#include \\"CudaFramework/CudaModern/CudaAlloc.hpp@g', 's@#include\\s*\\"CudaEvent\\.hpp@#include \\"CudaFramework/CudaModern/CudaEvent.hpp@g', 's@#include\\s*\\"CudaRefcounting\\.hpp@#include \\"CudaFramework/CudaModern/CudaRefcounting.hpp@g', 's@#include\\s*\\"CudaDeviceMemory\\.hpp@#include \\"CudaFramework/CudaModern/CudaDeviceMemory.hpp@g', 's@#include\\s*\\"CudaMatrixUtilities\\.hpp@#include \\"CudaFramework/CudaModern/CudaMatrixUtilities.hpp@g', 's@#include\\s*\\"CudaContextGroup\\.hpp@#include \\"CudaFramework/CudaModern/CudaContextGroup.hpp@g', 's@#include\\s*\\"CudaDeviceMatrix\\.hpp@#include \\"CudaFramework/CudaModern/CudaDeviceMatrix.hpp@g', 's@#include\\s*\\"CudaTypeDefs\\.hpp@#include \\"CudaFramework/CudaModern/CudaTypeDefs.hpp@g', 's@#include\\s*\\"CudaDeviceGroup\\.hpp@#include \\"CudaFramework/CudaModern/CudaDeviceGroup.hpp@g', 's@#include\\s*

In [18]:
commands = makeReplacementsIncludeGuard(dirs)
print(commands)
s = "\n".join(commands);
f = open("../ReplaceIncludeGuards.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(s);
f.close();

['fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef CudaFramework_CudaModern_CudaMatrix_hpp\\n#define CudaFramework_CudaModern_CudaMatrix_hpp@s" /home/zfmgpu/Desktop/Repository/CudaFramework/include/CudaFramework/CudaModern/CudaMatrix.hpp', 'fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef CudaFramework_CudaModern_CudaAlloc_hpp\\n#define CudaFramework_CudaModern_CudaAlloc_hpp@s" /home/zfmgpu/Desktop/Repository/CudaFramework/include/CudaFramework/CudaModern/CudaAlloc.hpp', 'fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef CudaFramework_CudaModern_CudaEvent_hpp\\n#define CudaFramework_CudaModern_CudaEvent_hpp@s" /home/zfmgpu/Desktop/Repository/CudaFramework/include/CudaFramework/CudaModern/CudaEvent.hpp', 'fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef CudaFramework_CudaModern_CudaRefcounting_hpp\\n#define CudaFramework_CudaModern_Cu